In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import datetime as dt
from collections import Counter
import re
import hashlib
import time

# called to counteract dark-mode Jupyter from swallowing axes &c.
plt.style.use('seaborn-whitegrid')

In [85]:
# open the pickle
df_brueter = pd.read_pickle('data/nabu_data2.pkl') 

In [86]:
# this is just for the file where I did not rename before pickling
# delete for final version
df_brueter.columns = ['web_id', 'bezirk', 'plz', 'ort', 'strasse', 'anhang', 'erstbeobachtung', 'beschreibung', 'besonderes', 'mauersegler',
            'kontrolle', 'sperling', 'ersatz', 'schwalbe', 'wichtig', 'star', 'sanierung', 'fledermaus', 'verloren', 'andere', 'hash']

In [87]:
df_brueter.head()

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,sperling,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash
0,1,Wilmersdorf,14197,Berlin,Eberbacher Str. 24,Skizze,10.07.2003,"2017: wie Vorjahre, jedoch neuer Standort link...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,1,0,0,0,0,1,0,0,0,352513ead3632be7e10e157f7f4412b0cbb16e030eadf2...
1,2,Wilmersdorf,14197,Berlin,Eberbacher Str. 18,,10.07.2003,"2012 wieder alle, dazu rechts von Haustür Mitt...",,1,...,1,0,0,0,0,0,0,0,0,a4b52b7da2e5560519d20194d5cbae277df57558e2e54f...
2,3,Wilmersdorf,14197,Berlin,Eberbacher Str. 30/Schlangenbader Str. 80,Skizzen und Fotos,10.07.2003,"2017: wie Vorjahre\r\n2014: 2x Mauersegler, di...","Sanierungsmaßnahme im Sommer bis Herbst 18, Fa...",1,...,1,0,0,1,0,0,0,0,0,199c11003859ef992f6718121436a47b10aaa6e54fcacd...
3,4,Wilmersdorf,14197,Berlin,Landauer Str. 2,,12.07.2003,Mauersegler und Sperlinge unter der Dachrinne ...,,1,...,1,0,0,0,0,0,0,0,0,44df52f1cd35a29a6d881fc8c7f987233355507a4af43e...
4,5,Wilmersdorf,14197,Berlin,Landauer Str. 9,,12.07.2003,2009: drei Mauerseglereinflüge und 2 Sperlings...,wichtiger Mauerseglerstandort,1,...,1,0,0,1,0,0,0,0,0,f19ca1f08a48a0d55021797808aa462ab869971126a509...


<hr>

Going along the columns, checking.

In [88]:
# 142 is still okay for a category
df_brueter.bezirk.nunique()

142

In [89]:
# I think we should get rid of the Steglitz-Zeh and Friedrichshai -like entries
df_brueter.bezirk.unique()

array(['Wilmersdorf', 'Köpenick', 'Schöneberg', 'Kreuzberg', 'Babelsberg',
       'Steglitz', 'Charlottenburg', 'Neukölln', 'Mitte',
       'Friedrichshain', 'Charlottenburg-Wilmersdorf, Schmargendorf',
       'Pankow', '', 'Tempelhof', 'Marienfelde', 'Reinickendorf',
       'Wannsee', 'Mariendorf', 'Spandau', 'Wedding',
       'Steglitz-Zehlendorf, Steglitz', 'Lankwitz',
       'Steglitz-Zehlendorf, Dahlem', 'Zehlendorf',
       'Steglitz-Zehlendorf, Lichterfelde-West', 'Prenzlauer Berg',
       'Lichtenberg', 'Moabit', 'Steglitz (Lichterfelde)',
       'Steglitz-Zehlendorf, Lichterfelde', 'Lichterfelde', 'Lichtfelde',
       'Steglitz-Zehlendorf, Lichterfelde-Ost', 'Steglitz-Zehl',
       ' Steglitz-Zeh', 'Steglitz-Zehlendorf', 'Neukölln, Buckow',
       'Lichtenberg, Friedrichsfelde', 'Wilmerdorf',
       'Steglitz (Lankwitz)', 'Mitte, Gesundbrunnen', 'Friedrichshai',
       'Tempelhof-Sch', 'Treptow', 'Wilmersdorf-Charlottenburg',
       'Charlottenburg-Wilmerdorf', 'Charlottenburg

In [90]:
bezirke_count = df_brueter.bezirk.value_counts()
bezirke_count

Steglitz                             472
Wilmersdorf                          464
Schöneberg                           298
Charlottenburg                       106
Lichtenberg                           82
                                    ... 
Steglitz / Lichterfelde                1
Charlottenburg / Grunewald             1
Steglitz-Zehlendorf, Wannsee           1
Lichterfelde, Steglitz-Zehlendorf      1
Steglitz-Zehlendorf                    1
Name: bezirk, Length: 142, dtype: int64

This is a bit of a mess. Could be solved by providing the user with a dropdown list of possible Bezirke or automatically entering the Bezirk according to the PLZ provided.

In [91]:
bezirke_count.to_csv('data/bezirke.csv', sep=',')

In [92]:
df_brueter.bezirk.isnull().values.any()

False

In [93]:
one_of_two = df_brueter.plz.loc[df_brueter['bezirk'] == 'Steglitz-Zehlendorf'].unique()
one_of_two

array(['12167', '12203', '12207', '12249', '12247', '12205', '12163',
       '14195', '14199', '12209', '12165', '14167', '14163', '14109',
       '14169'], dtype=object)

In [94]:
steglitz_plz = df_brueter.plz.loc[df_brueter['bezirk'] == 'Steglitz'].unique()
steglitz_plz

array(['14195', '12207', '12163', '12169', '12157', '12161', '12167',
       '12165', '12247', '12249', '14165', '12203', '12205', '12209',
       '10963', '14163', '12277', '14199'], dtype=object)

In [95]:
zehlendorf_plz = df_brueter.plz.loc[df_brueter['bezirk'] == 'Zehlendorf'].unique()
zehlendorf_plz

array(['14129', '14163', '14165', '14169', '14195', '14167'], dtype=object)

In [96]:
check_plz = np.intersect1d(zehlendorf_plz, steglitz_plz)
check_plz

array(['14163', '14165', '14195'], dtype=object)

Some PLZ are for example used for Steglitz and Zehlendorf. What could we do?

Maybe it is best just to remap the Bezirk based on the PLZ provided.

In [97]:
g = df_brueter.groupby('bezirk')['plz'].apply(lambda x: list(np.unique(x)))
g

bezirk
                                     [10781, 12159, 12207, 14513, 14797, 15569]
 Steglitz-Zeh                                                           [12207]
Alt-Treptow                                                             [12435]
Babelsberg                                                              [14482]
Berlin (Lankwitz)                                                       [12247]
                                                    ...                        
Weißensee/Heinersdorf                                                   [13086]
Wilmerdorf                                                              [14199]
Wilmersdorf                   [10587, 10623, 10625, 10629, 10707, 10709, 107...
Wilmersdorf-Charlottenburg                                       [10713, 10715]
Zehlendorf                           [14129, 14163, 14165, 14167, 14169, 14195]
Name: plz, Length: 142, dtype: object

In [98]:
g.to_csv('data/group_by_bezirk.csv', sep=',')

Postleitzahlen with Bezirksname from https://www.berlinstadtservice.de/xinh/Postleitzahlen_Berlin.html <br />
This was not as helpful as could be as some districts shared a PLZ and I would rather have one name for one PLZ district. But maybe that is not possible?

Yeah, it's not possible. <br />
>"Die Grenzen der Postleitzahlengebiete entsprechen nicht hundertprozentig den Bezirksgrenzen, so dass es bei einzelnen Postleitzahlen Überschneidungen mit anderen Bezirken geben kann." (Bezirksamt Charlottenburg-Wilmersdorf [https://www.berlin.de/ba-charlottenburg-wilmersdorf/service/charlottenburg-wilmersdorf-a-z/bezirk-a-z/artikel.299406.php])

It happens by more than "einzelnen Postleitzahlen". So, what to do now...<br />
I'll keep the original PLZ and amend any obvious errors in the Bezirk column. When mapping I can use the geoinformation for the PLZ areas provided by BerlinOpenData.

In [116]:
berlin_plz = {}
with open('data/berlin_plz.txt') as f:
    for line in f:
       (key, val) = line.split(' : ')
       berlin_plz[key] = val.strip()

berlin_plz

{'10585': 'Charlottenburg',
 '10587': 'Charlottenburg',
 '10589': 'Charlottenburg',
 '10623': 'Charlottenburg',
 '10625': 'Charlottenburg',
 '10627': 'Charlottenburg',
 '10629': 'Charlottenburg',
 '14050': 'Charlottenburg',
 '14052': 'Charlottenburg',
 '14053': 'Charlottenburg',
 '14057': 'Charlottenburg',
 '14059': 'Charlottenburg',
 '10713': 'Wilmersdorf',
 '10715': 'Wilmersdorf',
 '10717': 'Wilmersdorf',
 '14199': 'Grunewald / Schmargendorf / Wilmersdorf',
 '10711': 'Charlottenburg / Grunewald / Wilmersdorf',
 '14055': 'Charlottenburg / Grunewald / Wilmersdorf',
 '10707': 'Charlottenburg / Wilmersdorf',
 '10709': 'Charlottenburg / Wilmersdorf',
 '10719': 'Charlottenburg / Wilmersdorf',
 '10243': 'Friedrichshain',
 '10961': 'Kreuzberg',
 '10969': 'Kreuzberg',
 '10997': 'Kreuzberg',
 '10999': 'Kreuzberg',
 '13057': 'Falkenberg / Hohenschönhausen',
 '10315': 'Friedrichsfelde',
 '10319': 'Friedrichsfelde',
 '13053': 'Hohenschönhausen',
 '13055': 'Hohenschönhausen',
 '10318': 'Friedrichs

I'm going to leave the original bezirk as "gewachsene Struktur", as that is what the person reporting the sighting felt their kiez was called. I'm adding an "official" bezirk based on the PLZ.

In [120]:
# if PLZ value is a value in the dictionary replace Bezirk with key associated with the value
df_brueter['bezirk_neu'] = df_brueter['plz'].map(berlin_plz)

In [121]:
# Berlin has the PLZ 10115 to 14199
# Checking for records outside of this area

outside_berlin = df_brueter.loc[(df_brueter['plz'] < '10115') | (df_brueter['plz'] > '14199')]
outside_berlin

,web_id,bezirk,plz,ort,strasse,anhang,erstbeobachtung,beschreibung,besonderes,mauersegler,...,ersatz,schwalbe,wichtig,star,sanierung,fledermaus,verloren,andere,hash,bezirk_neu
41,43,Babelsberg,14482,Potsdam,Großbeerenstr. 171,,27.06.2004,2004-06: direkt gegenüber dem Eingang zum Film...,,1,...,0,0,0,0,0,0,0,0,61688eb7726ca8c8da7d9547436477b1cfd4753665b649...,NaN
235,240,,14513,Teltow,Heinersdorfer Weg 38 A-D,,15.08.2006,2006: zur Straße rechts an der Ecke unter der ...,,1,...,0,1,0,0,0,0,0,0,95d668101cf7971b8ecbfadf1ae3fc62caf4a0e08d68ea...,NaN
1489,1545,,15569,Woltersdorf,Schleusenstr. 33,,04.06.2016,2016: 5x Sperlinge über Dachrinne/Ziegelabschl...,,0,...,0,0,0,0,0,0,0,0,bab73792d13a408acac2053cb30f53e4fbdb3a2ced7d04...,NaN
1490,1546,,15569,Woltersdorf,Schleusenstr. 35a,,04.06.2016,2016: 4x Sperlinge unter Dachrinnenstück an li...,,0,...,0,0,0,0,0,0,0,0,dcb859b25e32724911c6af15a673a72d4d8b6067afb9ad...,NaN
1871,1927,Teltow-Fläming (Berliner Stadtgüter),14979,Großbeeren,"Osdorfer Str. 1, Gut Osdorf",Fotos,13.04.2020,"Gut Osdorf, Osdorfer Str. 1, 14979 Großbeeren ...",,0,...,0,1,1,0,0,0,0,0,ef4eeb98a808e1e4be2c8fbd35cf2762f2fd242740dce2...,NaN
2172,2228,Potsdam,14482,Potsdam,"Rudolf-Breitscheid-Str. 201, S-Bahnhof Griebni...",Protokoll,2021,2021-06: Rauchschwalben brüten\r\n,,0,...,0,1,0,0,0,0,0,0,e17c9b8c89182d166fd7d68c91b18213a218586ccef7eb...,NaN
2189,2245,,14797,Kloster Lehnin,"Friedensstr. 3, Kloster Lehnin",Foto,24.07.2021,2021-07: Im Klostergang ein Rauchschwalben-Nes...,,0,...,0,1,0,0,0,0,0,0,5a1e93015f4641826b7f3ba1a659d9a4a0f05e9a2c4fd9...,NaN


Okay, 6 different PLZ outside of Berlin is not too much, I'll add these by hand.